# Exploratory Text Analysis with Python

Workshop for the Southeast Data Librarian Symposium 2020

Scott Bailey <br/>
Digital Research and Scholarship Librarian <br/>
Copyright and Digital Scholarship Center <br/>
NC State University Libraries

## What do we mean by "exploratory text analysis?"

## A quick (!) overview of NLP libraries in Python
### Why spaCy?

## Jupyter Notebooks, Google Colab, and Binder



In [1]:
import spacy

In [2]:
nlp = spacy.load("en_core_web_md")

In [3]:
# from https://se-datalibrarian.github.io/2020/about/
sample_text = """The Southeast Data Librarian Symposium is intended to provide an opportunity for librarians and other research data specialists to explore developments in the field of data librarianship, including the management and sharing of research data.

In addition to learning about new work in the field, attendees will have the opportunity to network and build partnerships with regional colleagues. It is open to all who wish to attend, including students, data managers, and data scientists."""

In [4]:
doc = nlp(sample_text)

## Tokenization

In [5]:
for word in doc[:20]:
    print(word)

The
Southeast
Data
Librarian
Symposium
is
intended
to
provide
an
opportunity
for
librarians
and
other
research
data
specialists
to
explore


In [9]:
for noun_chunk in doc.noun_chunks:
    print(noun_chunk)

The Southeast Data Librarian Symposium
an opportunity
librarians
other research data specialists
developments
the field
data librarianship
the management
sharing
research data
addition
new work
the field
attendees
the opportunity
partnerships
regional colleagues
It
who
students
data managers
data scientists


In [6]:
for sent in doc.sents:
    print(sent)

The Southeast Data Librarian Symposium is intended to provide an opportunity for librarians and other research data specialists to explore developments in the field of data librarianship, including the management and sharing of research data.


In addition to learning about new work in the field, attendees will have the opportunity to network and build partnerships with regional colleagues.
It is open to all who wish to attend, including students, data managers, and data scientists.


## Part-of-speech tagging

In [7]:
for token in doc[:20]:
    print(token.text, token.pos_)

The DET
Southeast PROPN
Data PROPN
Librarian PROPN
Symposium PROPN
is AUX
intended VERB
to PART
provide VERB
an DET
opportunity NOUN
for ADP
librarians NOUN
and CCONJ
other ADJ
research NOUN
data NOUN
specialists NOUN
to PART
explore VERB


In [8]:
for token in doc[:20]:
    print(token.text, token.dep_)

The det
Southeast compound
Data compound
Librarian compound
Symposium nsubjpass
is auxpass
intended ROOT
to aux
provide xcomp
an det
opportunity dobj
for mark
librarians nsubj
and cc
other amod
research compound
data compound
specialists conj
to aux
explore advcl


### Visualization dependency tree

## Named entity recognition

### Visualizing named entities

## Word, sentence, and document vectors

In [ ]:
# similarity

## Cleaning text data

- stopwords
- lemmas
- punctuation
- alphanum
- lowercase

## Working with multiple documents (a corpus)

## Resources for spaCy

- [spaCy 101](https://spacy.io/usage/spacy-101)
- [Advanced NLP with spaCy](https://course.spacy.io/)